In [ ]:
%pip install openai

In [3]:
import os
from openai import AzureOpenAI

In [5]:

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview"
)

def call_llm(deployment_name, system_message, question):
    response = client.chat.completions.create(
        model = deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ]
    )
    print(response.choices[0].message.content)
    
deployment_name='gpt-4-32k'
# system_message = {"despojado":"Você é jum assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.",
#                   "formal":"Você é um assistente que sempre responde em tom formal, de forma didática e em português, sem citar palavras em inglês.",
#                   "informal":"Você é um assistente que sempre responde em tom informal, de forma didática e em português, com gírias e expressões coloquiais."}
# question = "O que é inteligência artificial?"

# for item in system_message:
#     print(item)
#     call_llm(deployment_name, system_message[item], "O que é inteligência artificial?")

agenda = ["12/03/2022 12:00:00 - Ocupado",
"12/03/2022 13:00:00 - Ocupado",
"12/03/2022 14:00:00 - Ocupado",
"12/03/2022 15:00:00 - Livre",
"12/03/2022 16:00:00 - Ocupado",
"12/03/2022 17:00:00 - Livre",
"12/03/2022 18:00:00 - Ocupado",
"12/03/2022 19:00:00 - Livre",
"12/03/2022 20:00:00 - Ocupado"]
call_llm(deployment_name, "Você é um bot que retorna as respostas em formato jSON", f"Com base nessa agenda, qual o próximo horário livre? {agenda}")


{
  "next_free_time": "12/03/2022 15:00:00"
}
